In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/Celerates/Notebook/Final Project/datasets/Resume.csv')
print(df.shape)
df.head(5)

(2484, 4)


,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2484 entries, 0 to 2483
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           2484 non-null   int64 
 1   Resume_str   2484 non-null   object
 2   Resume_html  2484 non-null   object
 3   Category     2484 non-null   object
dtypes: int64(1), object(3)
memory usage: 77.8+ KB


In [39]:
print("Missing values:\n", df.isnull().sum())

Missing values:
 ID             0
Resume_str     0
Resume_html    0
Category       0
dtype: int64


# preprocessing

### Text Cleaning

In [40]:
import re
import string
from bs4 import BeautifulSoup

corpus = df["Resume_str"].astype(str).tolist()

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\W', ' ', text)
    text = BeautifulSoup(text, "html.parser").get_text()
    text = re.sub(r'\s+', ' ', text).strip()
    return text
cleaned_corpus = [clean_text(doc) for doc in corpus]

In [41]:
df["cleaned_text"] = cleaned_corpus
print(df["cleaned_text"].head())

0    hr administratormarketing associate hr adminis...
1    hr specialist us hr operations summary versati...
2    hr director summary over years experience in r...
3    hr specialist summary dedicated driven and dyn...
4    hr manager skill highlights hr skills hr depar...
Name: cleaned_text, dtype: object


### Tokenization

In [ ]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt_tab')

tokenized_corpus = [word_tokenize(doc) for doc in cleaned_corpus]
print(tokenized_corpus)

### Stop Words Removal

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
filtered_corpus = [[word for word in doc if word not in stop_words] for doc in tokenized_corpus]
print(filtered_corpus)

### Stemming and Lemmatization

In [ ]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
nltk.download('wordnet')

stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

stemmed_corpus = [[stemmer.stem(word) for word in doc] for doc in filtered_corpus]
lemmatized_corpus = [[lemmatizer.lemmatize(word) for word in doc] for doc in filtered_corpus]
print(stemmed_corpus)
print(lemmatized_corpus)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
lemmatized_texts = [" ".join(tokens) for tokens in lemmatized_corpus]

tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
X = tfidf.fit_transform(lemmatized_texts)

print("TF-IDF shape:", X.shape)

TF-IDF shape: (2484, 5000)


### jumlah kategore (role)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(df["Category"])

print("Jumlah kategori:", len(le.classes_))

Jumlah kategori: 24


## TRAIN SPLIT

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train size:", X_train.shape)
print("Test size:", X_test.shape)

Train size: (1987, 5000)
Test size: (497, 5000)


## Build and Train Model (Naive Bayes)

### Terapkan SMOTE untuk Penanganan Ketidakseimbangan Kelas

In [ ]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)
X_train_smote, y_train_smote = sm.fit_resample(X_train, y_train)

print(f"Shape of X_train after SMOTE: {X_train_smote.shape}")
print(f"Shape of y_train after SMOTE: {y_train_smote.shape}")

Shape of X_train after SMOTE: (2304, 5000)
Shape of y_train after SMOTE: (2304,)


In [ ]:
from sklearn.naive_bayes import MultinomialNB

# Inisialisasi model Naive Bayes
nb_model_smote = MultinomialNB()

# Latih model menggunakan data yang sudah di-resample SMOTE
nb_model_smote.fit(X_train_smote, y_train_smote)

MultinomialNB()

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, accuracy_score, precision_score, recall_score, f1_score

y_pred = nb_model_smote.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

Accuracy: 0.5895372233400402

Classification Report:
                        precision    recall  f1-score   support

            ACCOUNTANT       0.51      0.88      0.65        24
              ADVOCATE       0.41      0.29      0.34        24
           AGRICULTURE       0.67      0.46      0.55        13
               APPAREL       0.60      0.16      0.25        19
                  ARTS       0.43      0.14      0.21        21
            AUTOMOBILE       0.31      0.57      0.40         7
              AVIATION       0.83      0.62      0.71        24
               BANKING       0.82      0.61      0.70        23
                   BPO       0.00      0.00      0.00         4
  BUSINESS-DEVELOPMENT       0.46      0.79      0.58        24
                  CHEF       0.81      0.71      0.76        24
          CONSTRUCTION       0.67      0.73      0.70        22
            CONSULTANT       0.67      0.09      0.15        23
              DESIGNER       0.80      0.57      0

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
accuracy = accuracy_score(y_test_smote, y_pred_smote)
precision = precision_score(y_test_smote, y_pred_smote)
recall = recall_score(y_test_smote, y_pred_smote)
f1 = f1_score(y_test_smote, y_pred_smote)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

NameError: name 'y_test_smote' is not defined

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Display confusion matrix
fig, ax = plt.subplots(figsize=(15, 15))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_)
disp.plot(cmap=plt.cm.Blues, ax=ax)
plt.title('Confusion Matrix')
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.show()